In [41]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')

## 0. Load Data

In [ ]:
data = pd.read_csv('../data/data.csv', error_bad_lines=False)
data.info()

## 1. Data Analysis: First Sight

### How is the date distribuited?

In [ ]:
flights_by_airline = data['OPERA'].value_counts()
plt.figure(figsize = (10, 2))
sns.set(style="darkgrid")
sns.barplot(x=flights_by_airline.index, y=flights_by_airline.values, alpha=0.9)
plt.title('Flights by Airline')
plt.ylabel('Flights', fontsize=12)
plt.xlabel('Airline', fontsize=12)
plt.xticks(rotation=90)
plt.show()

In [ ]:
flights_by_day = data['DIA'].value_counts()
plt.figure(figsize = (10, 2))
sns.set(style = "darkgrid")
sns.barplot(x=flights_by_day.index, y=flights_by_day.values, color='lightblue', alpha=0.8)
plt.title('Flights by Day')
plt.ylabel('Flights', fontsize=12)
plt.xlabel('Day', fontsize=12)
plt.xticks(rotation=90)
plt.show()

In [ ]:
flights_by_month = data['MES'].value_counts()
plt.figure(figsize = (10, 2))
sns.set(style = "darkgrid")
sns.barplot(x=flights_by_month.index, y=flights_by_month.values, color='lightblue', alpha=0.8)
plt.title('Flights by Month')
plt.ylabel('Flights', fontsize=12)
plt.xlabel('Month', fontsize=12)
plt.xticks(rotation=90)
plt.show()

In [ ]:
flights_by_day_in_week = data['DIANOM'].value_counts()
days = [
    flights_by_day_in_week.index[2], 
    flights_by_day_in_week.index[5], 
    flights_by_day_in_week.index[4], 
    flights_by_day_in_week.index[1], 
    flights_by_day_in_week.index[0], 
    flights_by_day_in_week.index[6], 
    flights_by_day_in_week.index[3]
]
values_by_day = [
    flights_by_day_in_week.values[2], 
    flights_by_day_in_week.values[5], 
    flights_by_day_in_week.values[4], 
    flights_by_day_in_week.values[1], 
    flights_by_day_in_week.values[0], 
    flights_by_day_in_week.values[6], 
    flights_by_day_in_week.values[3]
]
plt.figure(figsize = (10, 2))
sns.set(style="darkgrid")
sns.barplot(x=days, y=values_by_day, color='lightblue', alpha=0.8)
plt.title('Flights by Day in Week')
plt.ylabel('Flights', fontsize=12)
plt.xlabel('Day in Week', fontsize=12)
plt.xticks(rotation=90)
plt.show()

In [ ]:
flights_by_type = data['TIPOVUELO'].value_counts()
sns.set(style="darkgrid")
plt.figure(figsize = (10, 2))
sns.barplot(x= flights_by_type.index,y= flights_by_type.values, alpha=0.9)
plt.title('Flights by Type')
plt.ylabel('Flights', fontsize=12)
plt.xlabel('Type', fontsize=12)
plt.show()

In [ ]:
flight_by_destination = data['SIGLADES'].value_counts()
plt.figure(figsize = (10, 2))
sns.set(style="darkgrid")
sns.barplot(x= flight_by_destination.index,y= flight_by_destination.values, color = 'lightblue', alpha=0.8)
plt.title('Flight by Destination')
plt.ylabel('Flights', fontsize=12)
plt.xlabel('Destination', fontsize=12)
plt.xticks(rotation=90)

plt.show()

## 2. Features Generation

### 2.a. Period of Day

In [49]:
def get_period_day(dates):
    # Convert the series to datetime format if not already
    dates = pd.to_datetime(dates, errors='coerce')

    # Extract hour from dates
    hours = dates.dt.hour

    # Determine the period based on the hour
    conditions = [
        (hours >= 5) & (hours < 12),
        (hours >= 12) & (hours < 19)
    ]
    choices = ['mañana', 'tarde']

    return pd.Series(np.select(conditions, choices, default='noche'))

In [50]:
data['period_day'] = get_period_day(data['Fecha-I'])

### 2.b. High Season

In [51]:
def is_high_season(dates):
    # Convert the series to datetime if not already
    dates = pd.to_datetime(dates, errors='coerce')

    # Extract year from dates
    years = dates.dt.year

    # Define high season date ranges
    date_ranges = [
        ('12-15', '12-31'),
        ('01-01', '03-03'),
        ('07-15', '07-31'),
        ('09-11', '09-30')
    ]

    # Check for each range if dates fall into it
    masks = []
    for start, end in date_ranges:
        range_start = pd.to_datetime(years.astype(str) + '-' + start, errors='coerce')
        range_end = pd.to_datetime(years.astype(str) + '-' + end, errors='coerce')
        masks.append((dates >= range_start) & (dates <= range_end))

    # Combine all masks with 'or' operation
    final_mask = np.logical_or.reduce(masks)

    return final_mask.astype(int)

In [52]:
data['high_season'] = is_high_season(data['Fecha-I'])

### 2.c. Difference in Minutes

In [53]:
# Convert entire columns in `data` to datetime format; invalid parsing will produce NaT values
data['Fecha-O'] = pd.to_datetime(data['Fecha-O'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
data['Fecha-I'] = pd.to_datetime(data['Fecha-I'], format='%Y-%m-%d %H:%M:%S', errors='coerce')

# Calculate the time difference in minutes using vectorized operations
data['min_diff'] = (data['Fecha-O'] - data['Fecha-I']).dt.total_seconds() / 60

In [54]:
# data['min_diff'] = data.apply(get_min_diff, axis = 1)

### 2.d. Delay

In [55]:
threshold_in_minutes = 15
data['delay'] = np.where(data['min_diff'] > threshold_in_minutes, 1, 0)

In [ ]:
data.columns

## 3. Data Analysis: Second Sight

### How is the delay rate across columns?

In [57]:
def get_rate_from_column(data, column):
    # Count the delays for each unique value in the column
    delay_counts = data[data['delay'] == 1][column].value_counts()

    # Count the total occurrences for each unique value in the column
    total_counts = data[column].value_counts()

    # Calculate the rates
    rates = (total_counts / delay_counts.reindex(total_counts.index, fill_value=0)).fillna(0).round(2)

    return rates.reset_index().rename(columns={'index': column, column: 'Tasa (%)'})

In [ ]:
destination_rate = get_rate_from_column(data, 'SIGLADES')
destination_rate_values = data['SIGLADES'].value_counts().index
plt.figure(figsize = (20,5))
sns.set(style="darkgrid")
sns.barplot(x= destination_rate_values,y= destination_rate['Tasa (%)'], alpha = 0.75) ####
plt.title('Delay Rate by Destination')
plt.ylabel('Delay Rate [%]', fontsize=12)
plt.xlabel('Destination', fontsize=12)
plt.xticks(rotation=90)
plt.show()

In [ ]:
airlines_rate = get_rate_from_column(data, 'OPERA')
airlines_rate_values = data['OPERA'].value_counts().index
plt.figure(figsize = (20,5))
sns.set(style="darkgrid")
sns.barplot(x=airlines_rate_values,y= airlines_rate['Tasa (%)'], alpha = 0.75) ####
plt.title('Delay Rate by Airline')
plt.ylabel('Delay Rate [%]', fontsize=12)
plt.xlabel('Airline', fontsize=12)
plt.xticks(rotation=90)
plt.show()

In [ ]:
month_rate = get_rate_from_column(data, 'MES')
month_rate_value = data['MES'].value_counts().index
plt.figure(figsize = (20,5))
sns.set(style="darkgrid")
sns.barplot(x=month_rate_value,y= month_rate['Tasa (%)'], color = 'blue', alpha = 0.75) ###
plt.title('Delay Rate by Month')
plt.ylabel('Delay Rate [%]', fontsize=12)
plt.xlabel('Month', fontsize=12)
plt.xticks(rotation=90)
plt.ylim(0,10)
plt.show()

In [ ]:
days_rate = get_rate_from_column(data, 'DIANOM')
days_rate_value = data['DIANOM'].value_counts().index

sns.set(style="darkgrid")
plt.figure(figsize = (20, 5))
sns.barplot(x=days_rate_value,y= days_rate['Tasa (%)'], color = 'blue', alpha = 0.75) ####
plt.title('Delay Rate by Day')
plt.ylabel('Delay Rate [%]', fontsize=12)
plt.xlabel('Days', fontsize=12)
plt.xticks(rotation=90)
plt.ylim(0,7)
plt.show()

In [ ]:
high_season_rate = get_rate_from_column(data, 'high_season')
high_season_rate_values = data['high_season'].value_counts().index

plt.figure(figsize = (5, 2))
sns.set(style="darkgrid")
sns.barplot(x=["no", "yes"],y= high_season_rate['Tasa (%)']) ####
plt.title('Delay Rate by Season')
plt.ylabel('Delay Rate [%]', fontsize=12)
plt.xlabel('High Season', fontsize=12)
plt.xticks(rotation=90)
plt.ylim(0,6)
plt.show()

In [ ]:
flight_type_rate = get_rate_from_column(data, 'TIPOVUELO')
flight_type_rate_values = data['TIPOVUELO'].value_counts().index
plt.figure(figsize = (5, 2))
sns.set(style="darkgrid")
sns.barplot(x=flight_type_rate_values,y= flight_type_rate['Tasa (%)'])
plt.title('Delay Rate by Flight Type')
plt.ylabel('Delay Rate [%]', fontsize=12)
plt.xlabel('Flight Type', fontsize=12)
plt.ylim(0,7)
plt.show()

In [ ]:
period_day_rate = get_rate_from_column(data, 'period_day')
period_day_rate_values = data['period_day'].value_counts().index
plt.figure(figsize = (5, 2))
sns.set(style="darkgrid")
sns.barplot(x=period_day_rate_values,y= period_day_rate['Tasa (%)'])
plt.title('Delay Rate by Period of Day')
plt.ylabel('Delay Rate [%]', fontsize=12)
plt.xlabel('Period', fontsize=12)
plt.ylim(3,7)
plt.show()

## 4. Training

### 4.a. Data Split (Training and Validation)

In [65]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix, classification_report

In [66]:
training_data = shuffle(data[['OPERA', 'MES', 'TIPOVUELO', 'SIGLADES', 'DIANOM', 'delay']], random_state = 111)

In [67]:
features = pd.concat([
    pd.get_dummies(data['OPERA'], prefix = 'OPERA'),
    pd.get_dummies(data['TIPOVUELO'], prefix = 'TIPOVUELO'), 
    pd.get_dummies(data['MES'], prefix = 'MES')], 
    axis = 1
)
target = data['delay']

In [68]:
x_train, x_test, y_train, y_test = train_test_split(features, target, test_size=0.33, random_state=42)

In [ ]:
print(f"train shape: {x_train.shape} | test shape: {x_test.shape}")

In [ ]:
y_train.value_counts('%')*100

In [ ]:
y_test.value_counts('%')*100

### 4.b. Model Selection

#### 4.b.i. XGBoost

In [74]:
import xgboost as xgb
from xgboost import plot_importance

In [ ]:
xgb_model = xgb.XGBClassifier(random_state=1, learning_rate=0.01)
xgb_model.fit(x_train, y_train)

In [76]:
xgboost_y_preds = xgb_model.predict(x_test)
xgboost_y_preds = [1 if y_pred > 0.5 else 0 for y_pred in xgboost_y_preds]

In [ ]:
confusion_matrix(y_test, xgboost_y_preds)

In [ ]:
print(classification_report(y_test, xgboost_y_preds))

#### 4.b.ii. Logistic Regression

In [79]:
from sklearn.linear_model import LogisticRegression

In [ ]:
reg_model = LogisticRegression()
reg_model.fit(x_train, y_train)

In [81]:
reg_y_preds = reg_model.predict(x_test)

In [ ]:
confusion_matrix(y_test, reg_y_preds)

In [ ]:
print(classification_report(y_test, reg_y_preds))

## 5. Data Analysis: Third Sight

### Feature Importance

In [ ]:
plt.figure(figsize = (10,5))
plot_importance(xgb_model)

In [85]:
top_10_features = [
    "OPERA_Latin American Wings", 
    "MES_7",
    "MES_10",
    "OPERA_Grupo LATAM",
    "MES_12",
    "TIPOVUELO_I",
    "MES_4",
    "MES_11",
    "OPERA_Sky Airline",
    "OPERA_Copa Air"
]

### Data Balance

In [ ]:
n_y0 = len(y_train[y_train == 0])
n_y1 = len(y_train[y_train == 1])
scale = n_y0/n_y1
print(scale)

## 6. Training with Improvement

### 6.a. Data Split

In [ ]:
x_train2, x_test2, y_train2, y_test2 = train_test_split(features[top_10_features], target, test_size=0.33, random_state=42)

### 6.b. Model Selection

#### 6.b.i. XGBoost with Feature Importance and with Balance

In [ ]:
xgb_model_2 = xgb.XGBClassifier(random_state=1, learning_rate=0.01, scale_pos_weight = scale)
xgb_model_2.fit(x_train2, y_train2)

In [89]:
xgboost_y_preds_2 = xgb_model_2.predict(x_test2)

In [ ]:
confusion_matrix(y_test2, xgboost_y_preds_2)

In [ ]:
print(classification_report(y_test2, xgboost_y_preds_2))

#### 6.b.ii. XGBoost with Feature Importance but without Balance

In [ ]:
xgb_model_3 = xgb.XGBClassifier(random_state=1, learning_rate=0.01)
xgb_model_3.fit(x_train2, y_train2)

In [93]:
xgboost_y_preds_3 = xgb_model_3.predict(x_test2)

In [ ]:
confusion_matrix(y_test2, xgboost_y_preds_3)

In [ ]:
print(classification_report(y_test2, xgboost_y_preds_3))

#### 6.b.iii. Logistic Regression with Feature Importante and with Balance

In [ ]:
reg_model_2 = LogisticRegression(class_weight={1: n_y0/len(y_train), 0: n_y1/len(y_train)})
reg_model_2.fit(x_train2, y_train2)

In [97]:
reg_y_preds_2 = reg_model_2.predict(x_test2)

In [ ]:
confusion_matrix(y_test2, reg_y_preds_2)

In [ ]:
print(classification_report(y_test2, reg_y_preds_2))

#### 6.b.iv. Logistic Regression with Feature Importante but without Balance

In [ ]:
reg_model_3 = LogisticRegression()
reg_model_3.fit(x_train2, y_train2)

In [101]:
reg_y_preds_3 = reg_model_3.predict(x_test2)

In [ ]:
confusion_matrix(y_test2, reg_y_preds_3)

In [ ]:
print(classification_report(y_test2, reg_y_preds_3))

## 7. Data Science Conclusions

By looking at the results of the 6 trained models, it can be determined:
- There is no noticeable difference in results between XGBoost and LogisticRegression.
- Does not decrease the performance of the model by reducing the features to the 10 most important.
- Improves the model's performance when balancing classes, since it increases the recall of class "1".

**With this, the model to be productive must be the one that is trained with the top 10 features and class balancing, but which one?**